In [42]:
import glob
import pandas as pd
import os

import pyspark
import warnings
warnings.filterwarnings(action='ignore')


from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()

In [45]:
agg_df = spark.read.parquet('agg_df_y_m.parquet/')

In [46]:
agg_df.count()

1364775

In [47]:
df_pd = agg_df.toPandas()

In [48]:
df_pd.head(10)

,YEAR,MONTH,STATE_CITY_NAME,MAX_SNOW,MIN_SNOW,MAX_PRCP,MIN_PRCP,MEAN_SNOW,MEAN_PRCP,MEDIAN_SNOW,MEDIAN_PRCP,MEAN_LONG,MEAN_LAT
0,2015,01,AK CHULITNA RVR,305.0,0.0,152.0,0.0,11.483871,6.290323,0.0,0.0,-149.906100,62.825600
1,2015,01,AK Mt. Alyeska,0.0,0.0,356.0,0.0,0.000000,49.903226,0.0,0.0,-149.090000,60.960000
2,2015,01,AK SISIAK ALASKA,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,-160.395600,67.995000
3,2015,01,AR BULL SHOALS,0.0,0.0,538.0,0.0,0.000000,16.527273,0.0,0.0,-92.604385,36.369793
4,2015,01,CA BURBANK VALLEY PUMP PLT,0.0,0.0,170.0,0.0,0.000000,9.838710,0.0,0.0,-118.348100,34.186700
5,2015,01,CA READER RANCH CALIFORNIA,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,-121.117200,39.303600
6,2015,01,CA VAN NUYS AP,0.0,0.0,137.0,0.0,0.000000,9.258065,0.0,0.0,-118.489200,34.209700
7,2015,01,CO BERGEN PARK,89.0,8.0,66.0,5.0,33.600000,25.400000,20.0,18.0,-105.356500,39.708200
8,2015,01,CO FREDERICK,0.0,0.0,13.0,0.0,0.000000,1.290323,0.0,0.0,-104.974300,40.121300
9,2015,01,CO GOLDEN,198.0,0.0,97.0,0.0,8.462963,4.666667,0.0,0.0,-105.303791,39.792199


In [49]:
df_pd.to_csv('weather_data_year_month.csv')

In [14]:
lat_long_details = df_pd[['STATE_CITY_NAME','MEAN_LONG','MEAN_LAT']]

In [32]:
lat_long_details['STATE_CITY_NAME'].nunique()

20677

In [50]:
year_level= df_pd.groupby(by=['YEAR','STATE_CITY_NAME'],as_index=False).agg({'MAX_SNOW':'max',\
                                                                             'MIN_SNOW':'min',\
                                                                             'MAX_PRCP':'max',\
                                                                             'MIN_PRCP':'min',\
                                                                             'MEAN_SNOW':'mean',\
                                                                             'MEAN_PRCP':'mean',\
                                                                             'MEDIAN_SNOW':'median',\
                                                                             'MEDIAN_PRCP':'median',\
                                                                              'MEAN_LAT':'mean',\
                                                                            'MEAN_LONG':'mean',})

In [51]:
year_level.to_csv('agg_year.csv',index=False)

In [34]:
year_level.groupby(['STATE_CITY_NAME'],as_index=False).count()['YEAR']

,STATE_CITY_NAME,YEAR,MAX_SNOW,MIN_SNOW,MAX_PRCP,MIN_PRCP,MEAN_SNOW,MEAN_PRCP,MEDIAN_SNOW,MEDIAN_PRCP,MEAN_LAT,MEAN_LONG
0,AK ALCAN HWY MI-,7,7,7,7,7,7,7,7,7,7,7
1,AK ALPINE,7,7,7,7,7,7,7,7,7,7,7
2,AK ALPINE CREEK LODGE,3,3,3,3,3,3,3,3,3,3,3
3,AK ALYESKA,7,7,7,7,7,7,7,7,7,7,7
4,AK AMBER LAKE,7,7,7,7,7,7,7,7,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...
20672,WY YELLOWSTONE LAKE,7,7,7,7,7,7,7,7,7,7,7
20673,WY YELLOWSTONE NP E ENT,7,7,7,7,7,7,7,7,7,7,7
20674,WY YELLOWSTONE PARK MAMMOTH HCN,7,7,7,7,7,7,7,7,7,7,7
20675,WY YODER,7,7,7,7,7,7,7,7,7,7,7


In [52]:
year_level.loc[year_level['STATE_CITY_NAME']=='AK ALPINE CREEK LODGE']

,YEAR,STATE_CITY_NAME,MAX_SNOW,MIN_SNOW,MAX_PRCP,MIN_PRCP,MEAN_SNOW,MEAN_PRCP,MEDIAN_SNOW,MEDIAN_PRCP,MEAN_LAT,MEAN_LONG
2,2015,AK ALPINE CREEK LODGE,201.0,0.0,617.0,0.0,5.302560,11.810893,0.0,0.0,63.0428,-147.2478
17387,2016,AK ALPINE CREEK LODGE,305.0,0.0,838.0,0.0,2.608154,8.546505,0.0,0.0,63.0428,-147.2478
34884,2017,AK ALPINE CREEK LODGE,305.0,0.0,89.0,0.0,5.093030,2.194124,0.0,0.0,63.0428,-147.2478


In [37]:
year_level.loc[year_level['STATE_CITY_NAME']=='AK ALPINE']

,YEAR,STATE_CITY_NAME,MAX_SNOW,MIN_SNOW,MAX_PRCP,MIN_PRCP,MEAN_SNOW,MEAN_PRCP,MEDIAN_SNOW,MEDIAN_PRCP,MEAN_LAT,MEAN_LONG
1,2015,AK ALPINE,241.0,0.0,254.0,0.0,3.009594,3.009594,0.0,0.0,70.3464,-150.9283
17386,2016,AK ALPINE,46.0,0.0,226.0,0.0,1.254313,1.254313,0.0,0.0,70.3464,-150.9283
34883,2017,AK ALPINE,66.0,0.0,191.0,0.0,1.829781,1.829781,0.0,0.0,70.3464,-150.9283
52446,2018,AK ALPINE,76.0,0.0,152.0,0.0,1.508826,1.508826,0.0,0.0,70.3464,-150.9283
70044,2019,AK ALPINE,51.0,0.0,119.0,0.0,1.160721,1.160721,0.0,0.0,70.3464,-150.9283
87570,2020,AK ALPINE,58.0,0.0,130.0,0.0,1.181124,1.181124,0.0,0.0,70.3464,-150.9283
105259,2021,AK ALPINE,64.0,0.0,86.0,0.0,1.715591,1.715591,0.0,0.0,70.3464,-150.9283


In [54]:
year_level.to_csv('csv_year_level.csv',index=False)
df_pd.to_csv('csv_year_month_level.csv',index=False)